In [1]:
import onnx
from onnxscript import ir

import onnxdoctor
from onnxdoctor.diagnostics_providers import OnnxRuntimeCompatibilityLinter

In [2]:
proto = onnx.load("models/feastconv_Opset16.textproto")
proto = onnx.shape_inference.infer_shapes(proto)
model = ir.serde.deserialize_model(proto)

In [3]:
# The model should be compatible with ONNX Runtime
onnxdoctor.diagnose(model, [OnnxRuntimeCompatibilityLinter()])

[]

In [4]:
# Now build an invalid model

x = ir.Input(name="x", type=ir.TensorType(ir.DataType.FLOAT))
y = ir.Input(name="y", type=ir.TensorType(ir.DataType.INT64))
node = ir.Node("", "Add", (x, y))
graph = ir.Graph([x, y], node.outputs, nodes=[node], name="main_graph", opset_imports={"": 16})
model2 = ir.Model(graph, ir_version=8)
print(model2)

<
    ir_version=8,
    opset_imports={'': 16},
    producer_name=None,
    producer_version=None,
    domain=None,
    model_version=None,
>
graph(
    name=main_graph,
    inputs=(
        %"x"<FLOAT,?>,
        %"y"<INT64,?>
    ),
    outputs=(
        %"val_0"<?,?>
    ),
) {
    0 |  # node_Add_0
         %"val_0"<?,?> ⬅️ ::Add(%"x", %"y")
    return %"val_0"<?,?>
}


In [5]:
onnxdoctor.diagnose(model2, [OnnxRuntimeCompatibilityLinter(execution_provider="CUDAExecutionProvider")])

[DiagnosticsMessage(target_type='node', target=Node(name='node_Add_0', domain='', op_type='Add', inputs=(Input('x', type=Tensor(FLOAT), shape=None, producer=None, index=None), Input('y', type=Tensor(INT64), shape=None, producer=None, index=None)), attributes=OrderedDict(), overload='', outputs=(Value('val_0', type=None, shape=None, producer=node_Add_0, index=0),), version=None, doc_string=None), message='ONNX Runtime expects input y of operator ::Add to have type T=FLOAT (bounded by index 0), but found INT64.', severity='error', producer='OnnxRuntimeCompatibilityLinter', error_code='node-type-inconsistent')]

In [6]:
# Now build a model with a new opset

x = ir.Input(name="x", type=ir.TensorType(ir.DataType.FLOAT))
bias = ir.Input(name="y", type=ir.TensorType(ir.DataType.FLOAT))
scale = ir.Input(name="scale", type=ir.TensorType(ir.DataType.FLOAT))
node = ir.Node("", "GroupNormalization", (x, bias, scale), (ir.AttrInt64("group", 1), ir.AttrFloat32("epsilon", 1e-5)))
graph = ir.Graph([x, y], node.outputs, nodes=[node], name="main_graph", opset_imports={"": 21})
model3 = ir.Model(graph, ir_version=8)
print(model3)

<
    ir_version=8,
    opset_imports={'': 21},
    producer_name=None,
    producer_version=None,
    domain=None,
    model_version=None,
>
graph(
    name=main_graph,
    inputs=(
        %"x"<FLOAT,?>,
        %"y"<INT64,?>
    ),
    outputs=(
        %"val_0"<?,?>
    ),
) {
    0 |  # node_GroupNormalization_0
         %"val_0"<?,?> ⬅️ ::GroupNormalization(%"x", %"y", %"scale") {group=1, epsilon=1e-05}
    return %"val_0"<?,?>
}


In [7]:
onnxdoctor.diagnose(model3, [OnnxRuntimeCompatibilityLinter(execution_provider="CUDAExecutionProvider")])

[DiagnosticsMessage(target_type='node', target=Node(name='node_GroupNormalization_0', domain='', op_type='GroupNormalization', inputs=(Input('x', type=Tensor(FLOAT), shape=None, producer=None, index=None), Input('y', type=Tensor(FLOAT), shape=None, producer=None, index=None), Input('scale', type=Tensor(FLOAT), shape=None, producer=None, index=None)), attributes=OrderedDict({'group': AttrInt64('group', 1), 'epsilon': AttrFloat32('epsilon', 1e-05)}), overload='', outputs=(Value('val_0', type=None, shape=None, producer=node_GroupNormalization_0, index=0),), version=None, doc_string=None), message='Operator ::GroupNormalization not supported by CUDAExecutionProvider in ONNX Runtime.', severity='error', producer='OnnxRuntimeCompatibilityLinter', error_code='operator-unsupported')]

In [8]:
# But it does support com.microsoft::QuickGelu

x = ir.Input(name="x", type=ir.TensorType(ir.DataType.FLOAT))
node = ir.Node("com.microsoft", "QuickGelu", (x,))
graph = ir.Graph([x, y], node.outputs, nodes=[node], name="main_graph", opset_imports={"": 18, "com.microsoft": 1})
model4 = ir.Model(graph, ir_version=8)
print(model4)

<
    ir_version=8,
    opset_imports={'': 18, 'com.microsoft': 1},
    producer_name=None,
    producer_version=None,
    domain=None,
    model_version=None,
>
graph(
    name=main_graph,
    inputs=(
        %"x"<FLOAT,?>,
        %"y"<INT64,?>
    ),
    outputs=(
        %"val_0"<?,?>
    ),
) {
    0 |  # node_QuickGelu_0
         %"val_0"<?,?> ⬅️ com.microsoft::QuickGelu(%"x")
    return %"val_0"<?,?>
}


In [9]:
onnxdoctor.diagnose(model4, [OnnxRuntimeCompatibilityLinter()])

[]